In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time

##### 아래 URL의 NBA 데이터를 크롤링하여 판다스 데이터 프레임으로 나타내세요. 
- http://stats.nba.com/teams/traditional/?sort=GP&dir=-1

In [3]:
# 1.selenium을 사용한 방법

def NBA(url_path):
    print("Opening NBA webpage.",end="")
    driver = webdriver.Chrome()
    print(".",end="")
    driver.get(url_path)
    print(".",end="")
    driver.set_window_size(800,1000)
    print(".",end="")
    time.sleep(0.5)

    #컬럼데이터 불러오기
    print("Success!")
    time.sleep(0.5)
    print("Reading columns.",end="")
    columns = driver.find_elements_by_css_selector(".nba-stat-table > .nba-stat-table__overflow > table > thead > tr > th ")
    ls = []
    print("--> generate DataFrame columns...",end="")
    i = 0
    for cname in columns:
        tmp = cname.text
        ls.append(tmp)
        if i%5 == 0 : print(".",end="")
        i += 1
    df = pd.DataFrame(columns = ls[:28])
    print("Success!")
    time.sleep(0.5)
    print("Reading table data", end="")
    
    rows = driver.find_elements_by_css_selector(".nba-stat-table__overflow > table > tbody > tr ")
    
    print("--> Making DataFrame.",end="")
    #데이터를 DataFrame에 순차적으로 넣기
    for row in rows:
        row_datas = row.find_elements_by_css_selector("td")
        ls = []
        for data in row_datas:
            tem = data.text
            ls.append(tem)
        df.loc[len(df)] = ls[:28]
        print(".",end="")
    print("Success!")
    driver.quit()
    return df
    
url = 'http://stats.nba.com/teams/traditional/?sort=GP&dir=-1'    
NBA_df = NBA(url)
    
NBA_df

Opening NBA webpage....Success!
Reading columns.--> generate DataFrame columns..............Success!
Reading table data--> Making DataFrame.................Success!


,,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,1,Cleveland Cavaliers,22,12,10,.545,48.5,101.1,36.9,81.9,...,31.6,41.4,19.2,12.7,6.0,4.4,4.4,19.5,21.9,-1.7
1,2,Golden State Warriors,21,16,5,.762,48.2,110.4,41.2,86.0,...,35.9,45.0,26.0,12.9,7.9,5.8,3.6,19.7,19.4,10.0
2,3,Boston Celtics,19,11,8,.579,48.5,101.4,36.4,83.3,...,32.8,41.7,21.2,11.6,6.8,4.1,5.6,20.4,21.4,0.7
3,4,Houston Rockets,17,11,6,.647,48.0,104.9,37.2,85.3,...,34.2,43.0,18.8,12.1,8.1,5.5,4.4,19.1,19.5,1.8
4,5,Utah Jazz,11,5,6,.455,48.0,102.4,37.7,82.9,...,35.6,45.0,20.3,15.5,7.5,5.0,5.5,20.5,20.9,-2.2
5,6,Toronto Raptors,10,4,6,.400,48.5,107.3,38.9,82.3,...,30.7,40.1,21.8,14.0,5.7,6.1,4.8,22.6,20.3,-4.4
6,6,Philadelphia 76ers,10,5,5,.500,48.5,108.8,38.7,88.5,...,36.1,49.5,25.8,14.6,7.8,4.1,5.9,22.8,25.5,3.9
7,8,New Orleans Pelicans,9,5,4,.556,48.0,110.0,42.9,89.8,...,37.7,45.6,26.9,14.4,8.1,5.0,4.9,19.3,19.0,-0.8
8,9,Milwaukee Bucks,7,3,4,.429,48.7,101.9,39.1,78.3,...,33.0,38.6,22.0,13.1,6.6,6.7,4.1,22.9,19.4,-0.6
9,9,Indiana Pacers,7,3,4,.429,48.0,100.6,38.6,80.1,...,31.9,40.0,21.1,13.1,8.1,2.7,3.6,21.9,20.3,5.7


In [4]:
# 2.json을 이용한 방법(추천)
from pandas.io.json import json_normalize

def NBA_json(json_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'} 
    response = requests.get(json_url,  headers=headers)
    json_info = response.json()
    score_head = json_info["resultSets"][0]["headers"]
    score_table = json_info["resultSets"][0]["rowSet"]
    df = pd.DataFrame(score_table ,columns = score_head)
    
    df = df.sort_values(by=["GP","W","W_PCT"], ascending=False).reset_index(drop=True)
    
    return df

json_url = 'http://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision='
NBA_json(json_url)

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,1610612739,Cleveland Cavaliers,22,12,10,0.545,48.5,36.9,81.9,0.450,...,5,14,9,6,4,3,14,8,10,Cleveland Cavaliers
1,1610612744,Golden State Warriors,21,16,5,0.762,48.2,41.2,86.0,0.480,...,6,7,4,2,5,13,1,1,10,Golden State Warriors
2,1610612738,Boston Celtics,19,11,8,0.579,48.5,36.4,83.3,0.437,...,2,12,12,12,7,4,12,5,10,Boston Celtics
3,1610612745,Houston Rockets,17,11,6,0.647,48.0,37.2,85.3,0.436,...,3,5,5,5,2,11,7,4,10,Houston Rockets
4,1610612762,Utah Jazz,11,5,6,0.455,48.0,37.7,82.9,0.455,...,15,11,7,11,9,6,9,10,10,Utah Jazz
5,1610612755,Philadelphia 76ers,10,5,5,0.500,48.5,38.7,88.5,0.437,...,14,9,11,13,14,1,3,3,10,Philadelphia 76ers
6,1610612761,Toronto Raptors,10,4,6,0.400,48.5,38.9,82.3,0.473,...,10,15,3,7,13,8,4,12,10,Toronto Raptors
7,1610612740,New Orleans Pelicans,9,5,4,0.556,48.0,42.9,89.8,0.478,...,12,6,7,8,3,15,2,7,10,New Orleans Pelicans
8,1610612754,Indiana Pacers,7,3,4,0.429,48.0,38.6,80.1,0.481,...,7,4,16,1,12,9,15,2,10,Indiana Pacers
9,1610612749,Milwaukee Bucks,7,3,4,0.429,48.7,39.1,78.3,0.500,...,7,13,1,3,15,13,10,6,10,Milwaukee Bucks


##### 셀레니움을 이용하여 네이버 IT/과학 기사의 10 페이지 까지의 최신 제목 리스트를 크롤링하세요.
- http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105

In [3]:
# url이 최신기사를 가져오도록 속보기사의 URL사용
import datetime

# URL쿼리를 최신날짜와 입력된 페이지로 제조
def url_make(page=1): 
    now = datetime.datetime.now()
    nowDate = now.strftime('%Y%m%d')
    url_latest = 'http://news.naver.com/main/list.nhn?mode=LSD&sid1=105\
    &mid=sec&listType=title&date='+str(nowDate)+'&page='+str(page)
    
    return url_latest

# 입력된 페이지만큼의 최신기사를 가져오도록 함수설정
def naver_it_latest(pages = 10):
    print("Opening Chrome...",end="")

    driver = webdriver.Chrome()
    driver.set_window_size(800,1000)

    newses = []
#     df = pd.DataFrame(columns=["신문사","X분전","기사제목","link"])

    for i in range(1,pages+1):
        url = url_make(i)
        driver.get(url)
        print("Complete!\nPage",i,"scraping.",end="")
        lis = driver.find_elements_by_css_selector("ul.type02 > li")
        a = 0
        for li in lis:
            m_ago = li.find_element_by_css_selector(".date").text
            name = li.find_element_by_css_selector("a").text
            publisher = li.find_element_by_css_selector(".writing").text
            href = li.find_element_by_css_selector("a").get_attribute("href")
            if a%3 == 0 : print(".",end="")
            a += 1
            newses.append([publisher, m_ago, name, href])
#             df.loc[len(df)] = [publisher, m_ago, name, href]
    print("Complete!")
    df = pd.DataFrame(newses,columns=["신문사","X분전","기사제목","link"])
    print("Closing Chrome...")
    
    driver.quit()

    return df

naver_it_latest(3)

Opening Chrome...Complete!
Page 1 scraping..................Complete!
Page 2 scraping..................Complete!
Page 3 scraping..................Complete!
Closing Chrome...


,신문사,X분전,기사제목,link
0,ZDNet Korea,1분전,"웹젠, 포괄임금제 폐지...7월 1일부터 주 52시간 근무",http://news.naver.com/main/read.nhn?mode=LSD&m...
1,전자신문,1분전,"카카오M, 이병헌·공유·김태리 등 한류 연예기획사 품 안으로… 종합 콘텐츠 기업 도약",http://news.naver.com/main/read.nhn?mode=LSD&m...
2,조선비즈,2분전,"KT, 7월 2일부터 변경·해지한 이동전화 번호 재사용 금지",http://news.naver.com/main/read.nhn?mode=LSD&m...
3,아이뉴스24,2분전,"카카오M, 이병헌·공유·김태리 한류 스타군 확보",http://news.naver.com/main/read.nhn?mode=LSD&m...
4,아이뉴스24,2분전,"디블락, 오현석 파트너 대표이사로 전격 선임",http://news.naver.com/main/read.nhn?mode=LSD&m...
5,디지털타임스,3분전,"ETRI, 부산서 ‘해양·영상인식 기반 사업화 유망기술’ 선봬",http://news.naver.com/main/read.nhn?mode=LSD&m...
6,머니S,3분전,"애플, 아이폰6S로 인도 공략 가속… 현지만 출시할 듯",http://news.naver.com/main/read.nhn?mode=LSD&m...
7,한국일보,3분전,레진코믹스 5주년 기념 ‘135만 코인 쏜다’,http://news.naver.com/main/read.nhn?mode=LSD&m...
8,조선비즈,4분전,"HPE·삼성전자, CPU 의존도 낮춘 100배 빠른 '메모리 중심 컴퓨팅' 2년내 상용화",http://news.naver.com/main/read.nhn?mode=LSD&m...
9,디지털데일리,5분전,"이노그리드, 오픈 클라우드 플랫폼인 '파스-타(PaaS-TA)와 연동",http://news.naver.com/main/read.nhn?mode=LSD&m...
